In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from xgboost import XGBClassifier
from xgboost import XGBRegressor 

from sklearn.preprocessing import OrdinalEncoder
import datetime


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Datastorm/train_data.csv")
df1=pd.read_csv("/content/drive/MyDrive/Datastorm/validation_data.csv")

In [ ]:
df["WeeklySales"] = df["DailySales"]*7
df['Date'] = pd.to_datetime(df['DateID'])
df['year']= df['Date'].dt.year
df['week_of_year'] = df['Date'].dt.week
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


,CategoryCode,ItemCode,DateID,DailySales,WeeklySales,Date,year,week_of_year
0,category_2,117610,11/6/2021,7,49,2021-11-06,2021,44
1,category_4,836584,11/18/2021,16,112,2021-11-18,2021,46
2,category_1,370195,1/24/2022,6,42,2022-01-24,2022,4
3,category_2,172582,10/30/2021,5,35,2021-10-30,2021,43
4,category_2,1006009,10/30/2021,5,35,2021-10-30,2021,43
...,...,...,...,...,...,...,...,...
19916,category_2,225259,10/4/2021,4,28,2021-10-04,2021,40
19917,category_2,111436,10/12/2021,1,7,2021-10-12,2021,41
19918,category_2,1098502,10/1/2021,1,7,2021-10-01,2021,39
19919,category_2,20824,10/1/2021,7,49,2021-10-01,2021,39


In [ ]:
encoder = OrdinalEncoder()
scale_mapper = {"category_1":1, "category_2":2, "category_3":3,"category_4":4}
df["Category"] = df["CategoryCode"].replace(scale_mapper)
df

,CategoryCode,ItemCode,DateID,DailySales,WeeklySales,Date,year,week_of_year,Category,month
0,category_2,117610,11/6/2021,7,49,2021-11-06,2021,44,2,11
1,category_4,836584,11/18/2021,16,112,2021-11-18,2021,46,4,11
2,category_1,370195,1/24/2022,6,42,2022-01-24,2022,4,1,1
3,category_2,172582,10/30/2021,5,35,2021-10-30,2021,43,2,10
4,category_2,1006009,10/30/2021,5,35,2021-10-30,2021,43,2,10
...,...,...,...,...,...,...,...,...,...,...
19916,category_2,225259,10/4/2021,4,28,2021-10-04,2021,40,2,10
19917,category_2,111436,10/12/2021,1,7,2021-10-12,2021,41,2,10
19918,category_2,1098502,10/1/2021,1,7,2021-10-01,2021,39,2,10
19919,category_2,20824,10/1/2021,7,49,2021-10-01,2021,39,2,10


In [ ]:
df1["year"] = 2022
w1=datetime.date(2022,2,14).isocalendar()[1]
scale_mapper1 = {"w1":w1, "w2":w1+1, "w3":w1+2,"w4":w1+3}
df1["week_of_year"] = df1["Week"].replace(scale_mapper1)

In [ ]:
scale_mapper2 = {"category_1":1, "category_2":2, "category_3":3,"category_4":4}
df1["Category"] = df1["CategoryCode"].replace(scale_mapper2)
df1

,CategoryCode,ItemCode,Week,WeeklySales,year,week_of_year,Category,month
0,category_2,1044502,w1,11,2022,7,2,2
1,category_2,1105009,w1,11,2022,7,2,2
2,category_2,913561,w4,5,2022,10,2,4
3,category_1,1048975,w4,30,2022,10,1,4
4,category_1,17287,w2,60,2022,8,1,2
...,...,...,...,...,...,...,...,...
365,category_2,124954,w2,43,2022,8,2,2
366,category_2,40759,w1,48,2022,7,2,2
367,category_1,1090303,w1,19,2022,7,1,2
368,category_2,1090276,w3,6,2022,9,2,3


In [ ]:
train_x = df.drop(columns=['WeeklySales','DailySales','CategoryCode','Date','DateID','year'],axis=1)
train_y = df['WeeklySales']

In [ ]:
model = XGBClassifier()

# fit the model with the training data
model.fit(train_x,train_y)

XGBClassifier(objective='multi:softprob')

In [ ]:
test_x=df1.drop(columns=['WeeklySales','Week','CategoryCode','year'],axis=1)
test_y=df1["WeeklySales"]
Week=df1['Week']
predict_test = model.predict(test_x)
mean_absolute_percentage_error(test_y,predict_test)

0.7222711315631166

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
r2_score(test_y,predict_test)

-0.20419050344329626

In [ ]:
(abs(predict_test-test_y).sum())/(test_y.sum())


0.7986852917009039

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
r2_score(test_y1,predict_test1)

In [ ]:
(abs(predict_test1-test_y1).sum())/(test_y1.sum())

In [ ]:
df['WeeklySales']=df.groupby(['Category','ItemCode','week_of_year'])['DailySales'].transform(sum)
df

,CategoryCode,ItemCode,DateID,DailySales,WeeklySales,Date,year,week_of_year,Category,month
0,category_2,117610,11/6/2021,7,65,2021-11-06,2021,44,2,11
1,category_4,836584,11/18/2021,16,123,2021-11-18,2021,46,4,11
2,category_1,370195,1/24/2022,6,37,2022-01-24,2022,4,1,1
3,category_2,172582,10/30/2021,5,26,2021-10-30,2021,43,2,10
4,category_2,1006009,10/30/2021,5,13,2021-10-30,2021,43,2,10
...,...,...,...,...,...,...,...,...,...,...
19916,category_2,225259,10/4/2021,4,17,2021-10-04,2021,40,2,10
19917,category_2,111436,10/12/2021,1,11,2021-10-12,2021,41,2,10
19918,category_2,1098502,10/1/2021,1,3,2021-10-01,2021,39,2,10
19919,category_2,20824,10/1/2021,7,53,2021-10-01,2021,39,2,10


In [ ]:
df1


,CategoryCode,ItemCode,Week,WeeklySales,year,week_of_year,Category,month
0,category_2,1044502,w1,11,2022,7,2,2
1,category_2,1105009,w1,11,2022,7,2,2
2,category_2,913561,w4,5,2022,10,2,4
3,category_1,1048975,w4,30,2022,10,1,4
4,category_1,17287,w2,60,2022,8,1,2
...,...,...,...,...,...,...,...,...
365,category_2,124954,w2,43,2022,8,2,2
366,category_2,40759,w1,48,2022,7,2,2
367,category_1,1090303,w1,19,2022,7,1,2
368,category_2,1090276,w3,6,2022,9,2,3


In [ ]:
train_x2 = df.drop(columns=['WeeklySales','DailySales','CategoryCode','Date','DateID','year'],axis=1)
train_y2 = df['WeeklySales']
xgb_r = XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)
xgb_r.fit(train_x2, train_y2)
test_x2=df1.drop(columns=['WeeklySales','Week','CategoryCode','year'],axis=1)
test_y2=df1["WeeklySales"]
Week2=df1['Week']
pred = xgb_r.predict(test_x2)
mean_absolute_percentage_error(test_y2,pred)


[13:54:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


1.2579535139889972

In [ ]:
df2= pd.read_csv("/content/drive/MyDrive/Datastorm/test_data.csv")


In [ ]:
df2

In [ ]:
scale_mapper3 = {"category_1":1, "category_2":2, "category_3":3,"category_4":4}
df2["Category"] = df2["CategoryCode"].replace(scale_mapper3)
df2

In [ ]:
import datetime
w1=datetime.date(2022,2,14).isocalendar()[1]
scale_mapper4 = {"w1":w1, "w2":w1+1, "w3":w1+2,"w4":w1+3}
df2["week_of_year"] = df2["Week"].replace(scale_mapper4)

In [ ]:
df2

In [101]:
df2=df2.iloc[:, [0,1,2,3,5,4]]
test_x4=df2.drop(columns=['PredictedSales','Week','CategoryCode'],axis=1)
test_y4=df2["PredictedSales"]
Week=df2['Week']

predict_test = model.predict(test_x4)


NameError: ignored

In [ ]:
df2['WeelySales'] =predict_test

In [ ]:
df2['ID']=df2['CategoryCode']+'_'+df2['ItemCode'].astype(str) +'_'+ df2['Week']

In [ ]:
df2.drop(['CategoryCode'],axis=1,inplace=True)
df2.drop(['ItemCode'],axis=1,inplace=True)
df2.drop(['Week'],axis=1,inplace=True)
df2.drop(['PredictedSales'],axis=1,inplace=True)

In [ ]:
df2.drop(['week_of_year'],axis=1,inplace=True)
df2.drop(['Category'],axis=1,inplace=True)

In [ ]:
df2=df2.iloc[:, [1,0]]
df2

In [ ]:
from google.colab import files

df2.to_csv('submission2.csv',index=False)
files.download('submission2.csv')